In [3]:
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import fibsem
from fibsem.imaging import masks
from fibsem.imaging import utils   
from autoscript_sdb_microscope_client.structures import AdornedImage
from PIL import Image
import logging
import matplotlib.pyplot as plt
from scipy import ndimage as ndi
from skimage.segmentation import watershed
from skimage.feature import peak_local_max
from skimage import data
from skimage import filters
from skimage.color import rgb2gray
import imageio
import cv2
import glob
import itertools
import pandas as pd
from pprint import pprint
import plotly.express as px
import scipy
import statsmodels
import ipywidgets

In [3]:
def xcorr(
    img1: np.ndarray, 
    img2: np.ndarray, 
    use_mask: bool = True, 
    use_threshold: bool = True) -> np.ndarray:

    if img1.data.shape != img2.data.shape:
        err = f"Image 1 {img1.data.shape} and Image 2 {img2.data.shape} need to have the same shape"
        logging.error(err)
        raise ValueError(err)

    if use_mask:
        # Create Fourier Transform WITH Bandpass Filter
        pixelsize_img1 = img1.data.shape
        pixelsize_img2 = img2.data.shape
        img1_mask = masks.create_bandpass_mask(pixelsize_img1, 100, 4, 3)
        img2_mask = masks.create_bandpass_mask(pixelsize_img2, 100, 4, 3)
        img1fft = np.fft.ifftshift(img1_mask * np.fft.fftshift(np.fft.fft2(img1.data)))
        img2fft = np.fft.ifftshift(img2_mask * np.fft.fftshift(np.fft.fft2(img2.data)))
        assert img1_mask is not None, "Mask1 can not be None when using a Mask"
        assert img2_mask is not None, "Mask2 can not be None when using a Mask"

    else:
        # Create Fourier Transform
        img1fft = np.fft.ifftshift(np.fft.fftshift(np.fft.fft2(img1.data)))
        img2fft = np.fft.ifftshift(np.fft.fftshift(np.fft.fft2(img2.data)))

    # Do some shady normalization(?)
    n_pixels1 = img1.data.shape[0] * img1.data.shape[1]
    img1fft[0, 0] = 0
    tmp = img1fft * np. conj(img1fft)
    img1fft = n_pixels1 * img1fft / np.sqrt(tmp.sum())
    

    n_pixels2 = img2.data.shape[0] * img2.data.shape[1]
    img2fft[0, 0] = 0
    tmp = img2fft * np. conj(img2fft)
    img2fft = n_pixels2 * img2fft / np.sqrt(tmp.sum())


    if use_threshold:
        #Create threshold (Otsu's method) in abs Space
        abs1 = np.fft.fftshift(np.fft.ifftshift(np.fft.ifft2(img1fft)))
        abs2 = np.fft.fftshift(np.fft.ifftshift(np.fft.ifft2(img2fft)))
        otsu1 = filters.threshold_otsu(np.abs(abs1))
        otsu2 = filters.threshold_otsu(np.abs(abs2))

         #Create segmented binary Img
        binary1 = (abs1 > otsu1)*1
        binary2 = (abs2 > otsu2)*1
        
        #Back to Fourier Space for cross-correlation
        fft1 = np.fft.ifftshift(np.fft.fftshift(np.fft.fft2(binary1)))
        fft2 = np.fft.ifftshift(np.fft.fftshift(np.fft.fft2(binary2)))

        # Cross-correlate the two images
        corr = np.real(np.fft.fftshift(np.fft.ifft2(fft1 * np.conj(fft2))))

        #Cross-correlation center and shift from center
        maxX, maxY = np.unravel_index(np.argmax(corr), corr.shape)
        cen = np.asarray(corr.shape) / 2
        err = np.array(cen - [maxX, maxY], int)
        valMax = np.amax(corr)
        return corr, valMax, cen, err,

    else:
        # Cross-correlate the two images
        corr = np.real(np.fft.fftshift(np.fft.ifft2(img1fft * np.conj(img2fft))))
       
        #Cross-correlation center and shift from center
        maxX, maxY = np.unravel_index(np.argmax(corr), corr.shape)
        cen = np.asarray(corr.shape) / 2
        err = np.array(cen - [maxX, maxY], int)
        valMax = np.amax(corr)
        return corr, valMax, cen, err

In [4]:
data = []
files = glob.glob("../xcorrtest/*.tif")
for file in files:
    img = mpimg.imread(file)
    img = AdornedImage(img, None)
    data.append(img)
arr = np.array(data)

arr1 = arr[0: arr.size: 2]
arr2 = arr[1: arr.size: 2]

dict_list1 = []

for i in range(int(arr.size/2)):
    for use_mask in [True, False]:
        for use_threshold in [True, False]:
            corr1, maxVal1, cen1, err1 = xcorr(arr1[i], arr2[i], use_mask, use_threshold)
            data_dict1 = {
            "n":i, 
            "use_mask":use_mask, 
            "use_threshold":use_threshold,
            "corr1":corr1, 
            "maxVal1":maxVal1, 
            "cen1":cen1, 
            "err1_x":err1[0], 
            "err1_y":err1[1]}
            dict_list1.append(data_dict1)

df1 = pd.DataFrame.from_dict(dict_list1)

In [5]:
tilter = []
for i in range(int(arr.size/2)):
    cos = utils.cosine_stretch(arr2[i], 25)
    tilter.append(cos)
tilted = np.array(tilter)

dict_list2 = []

In [6]:

for i in range(int(arr.size/2)):
    for use_mask in [True, False]:
        for use_threshold in [True, False]:
            corr2, maxVal2, cen2, err2 = xcorr(arr1[i], tilted[i], use_mask, use_threshold)
            data_dict2 = {
            "n":i, 
            "use_mask":use_mask, 
            "use_threshold":use_threshold,
            "corr2":corr2, 
            "maxVal2":maxVal2, 
            "cen2":cen2, 
            "err2_x":err2[0], 
            "err2_y":err2[1]}
            dict_list2.append(data_dict2)

df2 = pd.DataFrame.from_dict(dict_list2)

In [49]:
df_n = df1[(df1["n"]==0)]
df_m_t = df1[(df1["use_mask"]==True) & (df1["use_threshold"]==True)]
df_m = df1[(df1["use_mask"]==True) & (df1["use_threshold"]==False)]
df_t = df1[(df1["use_mask"]==False) & (df1["use_threshold"]==True)]
df = df1[(df1["use_mask"]==False) & (df1["use_threshold"]==False)]

In [54]:
both_true = []
mask_true = []
thresh_true = []
both_false = []


for i, row in df_m_t.iterrows():
   temp1 = row["err1_x"], row["err1_y"]
   both_true.append(temp1)


for i, row in df_m.iterrows():
   temp2 = row["err1_x"], row["err1_y"]
   mask_true.append(temp2)
   

for i, row in df_t.iterrows():
   temp3 = row["err1_x"], row["err1_y"]
   thresh_true.append(temp3)


for i, row in df.iterrows():
   temp4 = row["err1_x"], row["err1_y"]
   both_false.append(temp4)




In [68]:
px.scatter(df_m_t, "n", ["err1_x", "err1_y"], color="use_mask", symbol="use_threshold")

In [69]:
px.scatter(df_m, "n", ["err1_x", "err1_y"], color="use_mask", symbol="use_threshold")

In [70]:
px.scatter(df_t, "n", ["err1_x", "err1_y"], color="use_mask", symbol="use_threshold")

In [71]:
px.scatter(df, "n", ["err1_x", "err1_y"], color="use_mask", symbol="use_threshold")